# Import des librairies

In [1]:
import numpy as np
import pandas as pd
from math import *
from sklearn.impute import KNNImputer

# 1 - Import et traitement de la table TRAIN

# I - Import de la table TRAIN

In [2]:
dt_train = pd.read_csv("Data_Base/application_train.csv", header = 0, sep = ',')

In [3]:
dt_train_copie = dt_train

In [4]:
dt_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# II - Analyse des données brutes

Pour l'analyse des données, on se base sur la table "train". Par la suite, on appliquera les mêmes traitement de nettoyage et de sélection des donénes à la table "test"

### Analyse des valeurs uniques de chaque colonnes

#### Analyse globale

In [5]:
dt_columns = dt_train.columns.values.tolist()

for i in dt_columns :
    print(f'colonne {i} : {dt_train[i].unique()}\n')

colonne SK_ID_CURR : [100002 100003 100004 ... 456253 456254 456255]

colonne TARGET : [1 0]

colonne NAME_CONTRACT_TYPE : ['Cash loans' 'Revolving loans']

colonne CODE_GENDER : ['M' 'F' 'XNA']

colonne FLAG_OWN_CAR : ['N' 'Y']

colonne FLAG_OWN_REALTY : ['Y' 'N']

colonne CNT_CHILDREN : [ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]

colonne AMT_INCOME_TOTAL : [202500.  270000.   67500.  ...  96768.  546250.5 113062.5]

colonne AMT_CREDIT : [ 406597.5 1293502.5  135000.  ...  181989.   743863.5 1391130. ]

colonne AMT_ANNUITY : [24700.5 35698.5  6750.  ... 71986.5 58770.  77809.5]

colonne AMT_GOODS_PRICE : [ 351000.  1129500.   135000.  ...  453465.   143977.5  743863.5]

colonne NAME_TYPE_SUITE : ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']

colonne NAME_INCOME_TYPE : ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']

colonne NAME_EDUCATION_TYPE : ['Secondary /

colonne ELEVATORS_MEDI : [0.   0.08  nan 0.16 0.4  0.14 0.28 0.04 0.24 0.88 0.32 0.12 0.36 0.68
 0.56 0.6  0.2  0.52 0.1  0.44 0.22 0.02 0.48 0.64 0.18 0.06 0.72 0.74
 0.9  0.5  0.42 1.   0.54 0.76 0.3  0.26 0.34 0.84 0.58 0.96 0.38 0.8
 0.92 0.78 0.66 0.46 0.62]

colonne ENTRANCES_MEDI : [0.069  0.0345    nan 0.2069 0.1379 0.1724 0.1034 0.1207 0.3103 0.4828
 0.3793 0.3448 0.2759 0.2414 0.1552 0.4483 0.0517 0.5172 0.1897 0.0862
 0.     0.4138 0.5517 0.2586 0.6207 0.6897 0.431  0.6552 0.5862 0.3276
 0.2241 0.7931 0.3621 0.8276 0.7586 0.4655 0.8621 0.7241 1.     0.2931
 0.8966 0.9655 0.5    0.931  0.569  0.3966 0.0172]

colonne FLOORSMAX_MEDI : [0.0833 0.2917    nan 0.1667 0.3333 0.6667 0.375  0.0417 0.25   0.4583
 0.2083 0.125  0.     0.5833 0.625  0.9167 0.9583 0.1875 0.5417 0.4792
 0.5    1.     0.4167 0.1042 0.2708 0.1458 0.4375 0.5208 0.875  0.2292
 0.7083 0.0208 0.3958 0.3542 0.6042 0.7292 0.8125 0.75   0.0625 0.5625
 0.3125 0.7917 0.6875 0.6458 0.8958 0.8333 0.7708 0.8542 0.9792 0

#### Analyse des "XNA"

In [6]:
columns_to_inspect = ['CODE_GENDER','ORGANIZATION_TYPE']

print(f'Nb XNA per column :\n')

for i in columns_to_inspect :
    l = dt_train[i].values.tolist()
    values_count = dict([(k, l.count(k)) for k in set(l)])
    nb = values_count['XNA']
    print(f'- colonne {i} : {nb}')

Nb XNA per column :

- colonne CODE_GENDER : 4
- colonne ORGANIZATION_TYPE : 55374


#### Analyse des "Unknown"

In [7]:
nb_unknown = 0

for i in range(len(dt_train['NAME_FAMILY_STATUS'])) :
    if dt_train['NAME_FAMILY_STATUS'][i] == 'Unknown' :
        nb_unknown += 1

print(f'La colonne "NAME_FAMILY_STATUS" contient {nb_unknown} "Unknown"')

La colonne "NAME_FAMILY_STATUS" contient 2 "Unknown"


#### Analyse des valeurs diverses

In [8]:
columns_to_inspect = ['OWN_CAR_AGE','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','CNT_FAM_MEMBERS']

print(f'Nb incompatible data per column :\n')

#for i in dt_columns_inspect :
  
analyse = dt_train[columns_to_inspect].describe()

for i in columns_to_inspect :
    
    print(f"{i} :")
    print(f"\t -> MIN : {analyse[i]['min']}")
    print(f"\t -> MAX : {analyse[i]['max']}")
    print(f"\n-------------------------\n")

Nb incompatible data per column :

OWN_CAR_AGE :
	 -> MIN : 0.0
	 -> MAX : 91.0

-------------------------

CNT_CHILDREN :
	 -> MIN : 0.0
	 -> MAX : 19.0

-------------------------

AMT_INCOME_TOTAL :
	 -> MIN : 25650.0
	 -> MAX : 117000000.0

-------------------------

AMT_CREDIT :
	 -> MIN : 45000.0
	 -> MAX : 4050000.0

-------------------------

AMT_ANNUITY :
	 -> MIN : 1615.5
	 -> MAX : 258025.5

-------------------------

AMT_GOODS_PRICE :
	 -> MIN : 40500.0
	 -> MAX : 4050000.0

-------------------------

DAYS_BIRTH :
	 -> MIN : -25229.0
	 -> MAX : -7489.0

-------------------------

DAYS_EMPLOYED :
	 -> MIN : -17912.0
	 -> MAX : 365243.0

-------------------------

DAYS_REGISTRATION :
	 -> MIN : -24672.0
	 -> MAX : 0.0

-------------------------

DAYS_ID_PUBLISH :
	 -> MIN : -7197.0
	 -> MAX : 0.0

-------------------------

CNT_FAM_MEMBERS :
	 -> MIN : 1.0
	 -> MAX : 20.0

-------------------------



#### Analyse des NaN

In [9]:
incomplet_data = 0

column_with_nan = []
tab_count_nan = []

for i in dt_columns :
    verif = dt_train[i].isna().sum()
    if verif != 0 :
        column_with_nan.append(i)
        tab_count_nan.append(dt_train[i].isna().sum())
        
    incomplet_data += dt_train[i].isna().sum()

nan_analysis = pd.DataFrame({'Feature' : column_with_nan ,  'Nb_NaN' : tab_count_nan})

print(f"{len(column_with_nan)} colonnes contiennet au moins un 'NaN' :")

nan_analysis

67 colonnes contiennet au moins un 'NaN' :


,Feature,Nb_NaN
0,AMT_ANNUITY,12
1,AMT_GOODS_PRICE,278
2,NAME_TYPE_SUITE,1292
3,OWN_CAR_AGE,202929
4,OCCUPATION_TYPE,96391
...,...,...
62,AMT_REQ_CREDIT_BUREAU_DAY,41519
63,AMT_REQ_CREDIT_BUREAU_WEEK,41519
64,AMT_REQ_CREDIT_BUREAU_MON,41519
65,AMT_REQ_CREDIT_BUREAU_QRT,41519


In [10]:
total_info = len(dt_train['SK_ID_CURR']) * len(dt_columns)

print(f"Total Data = {total_info}")
print(f"Missing Data = {incomplet_data}")
print(f"Missing Data Rate = {round( (incomplet_data / total_info),2) * 100} %")

Total Data = 37516342
Missing Data = 9152465
Missing Data Rate = 24.0 %


### Résultats de l'analyse :

- La colonne NAME_TYPE_SUITE contient des valeurs à unifier et d'autres à renomer :
    - Other_A & Other_B => Other
    - Spouse, partner => Partner


- La colonne NAME_EDUCATION_TYPE contient des valeurs à renomer :
    - Secondary / secondary special => Secondary


- La colonne NAME_FAMILY_STATUS contient des valeurs à renomer :
    - Single / not married => Single


- La colonne NAME_HOUSING_TYPE contient des valeurs à renomer :
    - House / Apartment => Housing owner


- La colonne ORGANIZATION_TYPE contient des valeurs à unifier :
    - Business Entity Type
    - Transport
    - Industry
    - Trade


- Les lignes contenant la valeurs 'XNA' / 'Unknown' sont à retirer des colonnes :
    - CODE_GENDER
    - NAME_FAMILY_STATUS
    
> Remarque : on ne peut pas déduire le sexe d'une personne ou sa situation familiale car les données que nous avons ne servent pas et ne permettent pas de déduire ce type d'information. On pourrait le faire mais le résultat sera non seulement non pertient mais cela voudrait en plus dire que le sexe et la situation d'une personne peuvent être définis par le niveau financier qu'elle détient.
    
- Les autres 'XNA' de la table sont à remplacer par des NaN


- Les valeurs extrêmes de la colonne DAYS_EMPLOYED sont à remplacer :
    - 365243 => NaN


- Créations de nouvelles colonnes :
    - TOTAL_AMT_CREDIT => somme totale qui doit être remboursée à la banque
    - PER_ANNUITY => Pourcentage de l'annuité dans la somme total à rembourser
    - CREDIT_PER_ON_GOODPRICE => Pourcentage de la part de somme empreinté pour l'achat du bien
    - YEARS_BIRTH => Age en années
    - PER_WORKED_DAYS => Pourcentage de jours d'occupation de l'emploie actuel sur le nombre total de jours vécus jusqu'à l'application du prêt
    - REGISTRATION_CHANGED => Si changement de l'inscription, vaut 1, sinon 0
    - ID_PUBLISH_CHANGED => Si changement de doc avant prêt, vaut 1, sinon 0
    - CAR_AGE_RANGE => Catégorie d'ancienneté du vahicule possédé
    - PHONE_CONTACTS => Nombre de numéro de téléphones fournis par le client
    - PHONE_CHANGED => Si changement de téléphone, vaut 1, sinon 0
    - ALL_DOCUMENTS_PROVIDED => 1 si dossier complet, sinon 0
    - TOTAL_AMT_REQ_CREDIT_BUREAU => Nombre de demandes de renseignement à propos du client, avant l'application du crédit


# III - Nettoyage des données

### Création de nouvelles colonnes

REAMRQUE : certaines colonnes ne sauront pas calculées avant d'avoir complété les données manquantes de la table

In [11]:
# YEARS_BIRTH : age en années
dt_train['YEARS_BIRTH'] = round((dt_train['DAYS_BIRTH'] / 365),0) * -1

# AGE_RANGE : age en années
dt_train['AGE_IN_YEARS_RANGE'] = pd.cut(x = dt_train['YEARS_BIRTH'], bins=[0, 25, 45, 65, 99], labels=[1, 2, 3, 4])

# PER_WORKED_DAYS : pourcentage de jours de sa vie passé à occuper le poste actuel
dt_train['PER_WORKED_DAYS'] = round(((dt_train['DAYS_EMPLOYED'] / dt_train['DAYS_BIRTH']) * 100),2)

# REGISTRATION_CHANGED : est ce qu'il y a eu changement de l'application choisi
dt_train['REGISTRATION_CHANGED'] = dt_train['DAYS_REGISTRATION'].apply(lambda x: 1 if x != 0 else 0)

# ID_PUBLISH_CHANGED : est ce qu'il y a eu changement de l'application choisi
dt_train['ID_PUBLISH_CHANGED'] = dt_train['DAYS_ID_PUBLISH'].apply(lambda x: 1 if x != 0 else 0)

# CAR_AGE_RANGE
dt_train['CAR_AGE_RANGE'] = pd.cut(x = dt_train['OWN_CAR_AGE'], bins=[0,10,20,91], labels=[1, 2, 3])

# PHONE_CONTACTS : nombre de num de tel renseignés
dt_train['PHONE_CONTACTS'] = dt_train['FLAG_MOBIL'] + dt_train['FLAG_EMP_PHONE'] + dt_train['FLAG_WORK_PHONE'] + dt_train['FLAG_PHONE'] + dt_train['FLAG_EMAIL']

# PHONE_CHANGED : y a-t-il eu changement de tel
dt_train['PHONE_CHANGED'] = dt_train['DAYS_LAST_PHONE_CHANGE'].apply(lambda x: 1 if x != 0 else 0)

# ALL_DOCUMENTS_PROVIDED
dt_train['Nb_Documents_provided'] = dt_train['FLAG_DOCUMENT_2'] + dt_train['FLAG_DOCUMENT_3'] + dt_train['FLAG_DOCUMENT_4'] + dt_train['FLAG_DOCUMENT_5'] + dt_train['FLAG_DOCUMENT_6'] + dt_train['FLAG_DOCUMENT_7'] + dt_train['FLAG_DOCUMENT_8'] + dt_train['FLAG_DOCUMENT_9'] + dt_train['FLAG_DOCUMENT_10'] + dt_train['FLAG_DOCUMENT_11'] + dt_train['FLAG_DOCUMENT_12'] + dt_train['FLAG_DOCUMENT_13'] + dt_train['FLAG_DOCUMENT_14'] + dt_train['FLAG_DOCUMENT_15'] + dt_train['FLAG_DOCUMENT_16'] + dt_train['FLAG_DOCUMENT_17'] + dt_train['FLAG_DOCUMENT_18'] + dt_train['FLAG_DOCUMENT_19'] + dt_train['FLAG_DOCUMENT_20'] + dt_train['FLAG_DOCUMENT_21']
dt_train['DOCUMENTS_PROVIDED'] = dt_train['Nb_Documents_provided'].apply(lambda x: 1 if x != 0 else 0)

### Retrait des lignes inutiles

In [12]:
dt_train = dt_train[dt_train['CODE_GENDER'] != 'XNA']
dt_train = dt_train[dt_train['NAME_FAMILY_STATUS'] != 'Unknown']

### Réctification des valeurs incorrectes

In [13]:
# NAME_TYPE_SUITE
dt_train['NAME_TYPE_SUITE']=np.where(dt_train['NAME_TYPE_SUITE'] == 'Other_A', 'Other', dt_train['NAME_TYPE_SUITE'])
dt_train['NAME_TYPE_SUITE']=np.where(dt_train['NAME_TYPE_SUITE'] == 'Other_B', 'Other', dt_train['NAME_TYPE_SUITE'])
dt_train['NAME_TYPE_SUITE']=np.where(dt_train['NAME_TYPE_SUITE'] == 'Spouse, partner', 'Partner', dt_train['NAME_TYPE_SUITE'])

# NAME_EDUCATION_TYPE
dt_train['NAME_EDUCATION_TYPE']=np.where(dt_train['NAME_EDUCATION_TYPE'] == 'Secondary / secondary special', 'Secondary', dt_train['NAME_EDUCATION_TYPE'])

# NAME_FAMILY_STATUS
dt_train['NAME_FAMILY_STATUS']=np.where(dt_train['NAME_FAMILY_STATUS'] == 'Single / not married', 'Single', dt_train['NAME_FAMILY_STATUS'])

# NAME_HOUSING_TYPE
dt_train['NAME_HOUSING_TYPE']=np.where(dt_train['NAME_HOUSING_TYPE'] == 'House / apartment', 'Housing owner', dt_train['NAME_HOUSING_TYPE'])

# ORGANIZATION_TYPE    
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Business Entity Type 1', 'Business', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Business Entity Type 2', 'Business', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Business Entity Type 3', 'Business', dt_train['ORGANIZATION_TYPE'])

dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Transport: type 1', 'Transport', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Transport: type 2', 'Transport', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Transport: type 3', 'Transport', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Transport: type 4', 'Transport', dt_train['ORGANIZATION_TYPE'])

dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 1', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 2', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 3', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 4', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 5', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 6', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 7', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 8', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 9', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 10', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 11', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 12', 'Industry', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Industry: type 13', 'Industry', dt_train['ORGANIZATION_TYPE'])

dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 1', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 2', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 3', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 4', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 5', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 6', 'Trade', dt_train['ORGANIZATION_TYPE'])
dt_train['ORGANIZATION_TYPE']=np.where(dt_train['ORGANIZATION_TYPE'] == 'Trade: type 7', 'Trade', dt_train['ORGANIZATION_TYPE'])
    
# DAYS_EMPLOYED
dt_train['DAYS_EMPLOYED']=np.where(dt_train['DAYS_EMPLOYED'] == 365243, np.nan, dt_train['DAYS_EMPLOYED'])

### Retrait des colonnes inutiles

In [14]:
dt_train = dt_train.drop(['REGION_RATING_CLIENT',
'DAYS_BIRTH','HOUR_APPR_PROCESS_START','DAYS_REGISTRATION',
'APARTMENTS_AVG','DAYS_ID_PUBLISH',
'BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BUILD_AVG','COMMONAREA_AVG',
'OWN_CAR_AGE','ELEVATORS_AVG','ENTRANCES_AVG',
'FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG',
'LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG',
'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG',
'APARTMENTS_MODE','FLAG_MOBIL','FLAG_EMP_PHONE',
'FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL',
'BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE',
'COMMONAREA_MODE','ELEVATORS_MODE','ENTRANCES_MODE',
'FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE',
'LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI',
'YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI',
'ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI',
'LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI',
'NONLIVINGAREA_MEDI','FONDKAPREMONT_MODE','HOUSETYPE_MODE','TOTALAREA_MODE','WALLSMATERIAL_MODE',
'EMERGENCYSTATE_MODE','DAYS_LAST_PHONE_CHANGE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9',
'FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15',
'FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21',
'Nb_Documents_provided'], axis = 1)

### Convertions valeurs

#### Convertion valeurs globales

In [15]:
# XNA => NaN
dt_train = dt_train.replace('XNA', np.nan)

# Y => 1
dt_train = dt_train.replace('Y', 1)

# N => 0
dt_train = dt_train.replace('N', 0)

#### Convertion valeurs spécifiques aux colonnes

##### Détection des colonnes à modifer

In [16]:
# récupération des colonnes à reformer
colonne_reform = []

columnes_with_str = dt_train.columns

for i in columnes_with_str :
    cran = 1
    while cran != 0 :
        if dt_train[i][0] != np.nan :
            cran = 0
            if type(dt_train[i][0]) == str :
                colonne_reform.append(i)

colonne_reform

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE']

In [17]:
longest_list = 0


# récupération du plus grand nombre de valeurs uniques présent dans une colonne 
for i in colonne_reform :
    if len(dt_train[i].unique()) > longest_list :
        longest_list = len(dt_train[i].unique())


# création d'un tableau de sauvegarde des colonnes contenant des valeurs uniques de type "chaine de caractères"      
values_save = [[''] * (longest_list+1) for x in range(len(colonne_reform))]

# remplissage du tableau de sauvegarde
for i in range (len(colonne_reform)) :
    
    values_save[i][0] = colonne_reform[i]
    
    valeurs_uniques = dt_train[colonne_reform[i]].unique()
    
    j = 1
    diff = 1
        
    for j in range (len(valeurs_uniques)) :
   
        if type(valeurs_uniques[j]) == str :
            values_save[i][j+diff] = valeurs_uniques[j]
        else :
            diff -= 1
        

values_save

[['NAME_CONTRACT_TYPE',
  'Cash loans',
  'Revolving loans',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['CODE_GENDER',
  'M',
  'F',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['NAME_TYPE_SUITE',
  'Unaccompanied',
  'Family',
  'Partner',
  'Children',
  'Other',
  'Group of people',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['NAME_INCOME_TYPE',
  'Working',
  'State servant',
  'Commercial associate',
  'Pensioner',
  'Unemployed',
  'Student',
  'Businessman',
  'Maternity leave',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

##### Transformation des chaines de caractère en équivalent de type integer

In [18]:
# [i][0] ==> Colonnes

for i in range (len(values_save)) :
    for j in range (len(values_save[i])) :
        dt_train[values_save[i][0]] = dt_train[values_save[i][0]].replace(values_save[i][j], j)

In [19]:
dt_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_YEAR,YEARS_BIRTH,AGE_IN_YEARS_RANGE,PER_WORKED_DAYS,REGISTRATION_CHANGED,ID_PUBLISH_CHANGED,CAR_AGE_RANGE,PHONE_CONTACTS,PHONE_CHANGED,DOCUMENTS_PROVIDED
0,100002,1,1,1,0,1,0,202500.0,406597.5,24700.5,...,1.0,26.0,2,6.73,1,1,NaN,3,1,1
1,100003,0,1,2,0,0,0,270000.0,1293502.5,35698.5,...,0.0,46.0,3,7.09,1,1,NaN,3,1,1
2,100004,0,2,1,1,1,0,67500.0,135000.0,6750.0,...,0.0,52.0,3,1.18,1,1,3,4,1,0
3,100006,0,1,2,0,1,0,135000.0,312682.5,29686.5,...,NaN,52.0,3,15.99,1,1,NaN,2,1,1
4,100007,0,1,1,0,1,0,121500.0,513000.0,21865.5,...,0.0,55.0,3,15.24,1,1,NaN,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,1,1,0,0,0,157500.0,254700.0,27558.0,...,NaN,26.0,2,2.53,1,1,NaN,2,1,1
307507,456252,0,1,2,0,1,0,72000.0,269550.0,12001.5,...,NaN,57.0,3,-1758.09,1,1,NaN,2,0,1
307508,456253,0,1,2,0,1,0,153000.0,677664.0,29979.0,...,1.0,41.0,2,52.93,1,1,NaN,3,1,1
307509,456254,1,1,2,0,1,0,171000.0,370107.0,20205.0,...,0.0,33.0,2,40.01,1,1,NaN,2,1,1


### Analyse des données après nettoyage

Nous remarquons que nos étapes de distilation des données a été utile parce que nous avons une donnée plus pure, plus pertinente, avec moins de données manquantes.
Cela va donc nous permettre de compléter les missing datas avec des information beaucoup plus pertinentes !

In [20]:
dt_columns = dt_train.columns

incomplet_data = 0

column_with_nan = []
tab_count_nan = []

for i in dt_columns :
    verif = dt_train[i].isna().sum()
    if verif != 0 :
        column_with_nan.append(i)
        tab_count_nan.append(dt_train[i].isna().sum())
        
    incomplet_data += dt_train[i].isna().sum()

nan_analysis = pd.DataFrame({'Feature' : column_with_nan ,  'Nb_NaN' : tab_count_nan})

print(f"{len(column_with_nan)} colonnes contiennet au moins un 'NaN' :")

nan_analysis

20 colonnes contiennet au moins un 'NaN' :


,Feature,Nb_NaN
0,AMT_ANNUITY,12
1,AMT_GOODS_PRICE,276
2,NAME_TYPE_SUITE,1290
3,DAYS_EMPLOYED,55374
4,OCCUPATION_TYPE,96389
5,ORGANIZATION_TYPE,55374
6,EXT_SOURCE_1,173375
7,EXT_SOURCE_2,660
8,EXT_SOURCE_3,60964
9,OBS_30_CNT_SOCIAL_CIRCLE,1021


In [21]:
total_info = len(dt_train['SK_ID_CURR']) * len(dt_columns)

print(f"Total Data = {total_info}")
print(f"Missing Data = {incomplet_data}")
print(f"Missing Data Rate = {round( (incomplet_data / total_info),2) * 100} %")

Total Data = 15990260
Missing Data = 901965
Missing Data Rate = 6.0 %


### Traitement des NaN

#### Récupération des colonnes contenant des NaN

In [22]:
columns_with_NA = []
for i in nan_analysis['Feature'] :
    columns_with_NA.append(i)
    
X_with_NaN = dt_train[columns_with_NA]

#### Calcul des NaN via le KNNImputer

In [23]:
nb_iteration = len(X_with_NaN) / 5000

if float(nb_iteration).is_integer() == False :
    nb_iteration = ceil(nb_iteration)
    last_round_incomplet = True
    
df_result = 0
v = 0
s = 0

print(nb_iteration)

while v < (nb_iteration-1) :
    
    f = s + 5000
    
    if v < (nb_iteration-2) :
        
        X_with_NaN_used = X_with_NaN[s:f]
        
        imputer = KNNImputer(n_neighbors = 15)
        
        X_without_nan = imputer.fit_transform(X_with_NaN_used)
        
        X_df_result = pd.DataFrame(X_without_nan,columns=X_with_NaN_used.columns)
        
        if v == 0 :
            df_result = X_df_result
        
        if v > 0 :
            df_result = pd.concat([df_result,X_df_result], ignore_index=True)
        
        print(f'==> Round {v} completed !!!')
        
        v += 1    
        
    else :
        
        X_with_NaN_used = X_with_NaN[s:]
        
        imputer = KNNImputer(n_neighbors = 15)
  
        X_without_nan = imputer.fit_transform(X_with_NaN_used)
        
        X_df_result = pd.DataFrame(X_without_nan,columns=X_with_NaN_used.columns)
        
        df_result = pd.concat([df_result,X_df_result], ignore_index=True)
        
        v += 1
        
        print('==> Last round completed !!!')
        
    if v < (nb_iteration-1) :
        s = f 

62
==> Round 0 completed !!!
==> Round 1 completed !!!
==> Round 2 completed !!!
==> Round 3 completed !!!
==> Round 4 completed !!!
==> Round 5 completed !!!
==> Round 6 completed !!!
==> Round 7 completed !!!
==> Round 8 completed !!!
==> Round 9 completed !!!
==> Round 10 completed !!!
==> Round 11 completed !!!
==> Round 12 completed !!!
==> Round 13 completed !!!
==> Round 14 completed !!!
==> Round 15 completed !!!
==> Round 16 completed !!!
==> Round 17 completed !!!
==> Round 18 completed !!!
==> Round 19 completed !!!
==> Round 20 completed !!!
==> Round 21 completed !!!
==> Round 22 completed !!!
==> Round 23 completed !!!
==> Round 24 completed !!!
==> Round 25 completed !!!
==> Round 26 completed !!!
==> Round 27 completed !!!
==> Round 28 completed !!!
==> Round 29 completed !!!
==> Round 30 completed !!!
==> Round 31 completed !!!
==> Round 32 completed !!!
==> Round 33 completed !!!
==> Round 34 completed !!!
==> Round 35 completed !!!
==> Round 36 completed !!!
==> Roun

In [24]:
df_result

,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,DAYS_EMPLOYED,OCCUPATION_TYPE,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CAR_AGE_RANGE
0,24700.5,351000.0,1.0,-637.0,1.000000,1.000000,0.083037,0.262949,0.139376,2.0,2.0,2.0,2.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.800000
1,35698.5,1129500.0,2.0,-1188.0,2.000000,2.000000,0.311267,0.622246,0.548131,1.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.666667
2,6750.0,135000.0,1.0,-225.0,1.000000,3.000000,0.461932,0.555912,0.729567,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.000000
3,29686.5,297000.0,1.0,-3039.0,1.000000,1.000000,0.449071,0.650442,0.506965,2.0,0.0,2.0,0.0,0.066667,0.0,0.066667,0.066667,0.200000,1.866667,1.733333
4,21865.5,513000.0,1.0,-3038.0,2.000000,4.000000,0.543744,0.322738,0.452414,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307500,27558.0,225000.0,1.0,-236.0,6.000000,16.000000,0.145570,0.681632,0.540520,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.066667,1.266667,1.400000
307501,12001.5,225000.0,1.0,-2267.4,5.133333,6.733333,0.486282,0.115992,0.448786,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.133333,0.400000,2.000000,1.400000
307502,29979.0,585000.0,1.0,-7921.0,4.000000,2.000000,0.744026,0.535722,0.218859,6.0,0.0,6.0,0.0,1.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.533333
307503,20205.0,319500.0,1.0,-4786.0,1.000000,1.000000,0.451627,0.514163,0.661024,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.533333


#### Traitement du résultat obtenu pour les NaN

In [25]:
# OCCUPATION_TYPE => arrondir à l'entier le plus proche
df_result['OCCUPATION_TYPE'] = round(df_result['OCCUPATION_TYPE'])

# NAME_TYPE_SUITE => arrondir à l'entier le plus proche
df_result['NAME_TYPE_SUITE'] = round(df_result['NAME_TYPE_SUITE'])

# ORGANIZATION_TYPE => arrondir à l'entier le plus proche
df_result['ORGANIZATION_TYPE'] = round(df_result['ORGANIZATION_TYPE'])

# OBS_30_CNT_SOCIAL_CIRCLE => arrondir à l'entier le plus proche
df_result['OBS_30_CNT_SOCIAL_CIRCLE'] = round(df_result['OBS_30_CNT_SOCIAL_CIRCLE'])

# DEF_30_CNT_SOCIAL_CIRCLE => arrondir à l'entier le plus proche
df_result['DEF_30_CNT_SOCIAL_CIRCLE'] = round(df_result['DEF_30_CNT_SOCIAL_CIRCLE'])

# OBS_60_CNT_SOCIAL_CIRCLE => arrondir à l'entier le plus proche
df_result['OBS_60_CNT_SOCIAL_CIRCLE'] = round(df_result['OBS_60_CNT_SOCIAL_CIRCLE'])

# DEF_60_CNT_SOCIAL_CIRCLE => arrondir à l'entier le plus proche
df_result['DEF_60_CNT_SOCIAL_CIRCLE'] = round(df_result['DEF_60_CNT_SOCIAL_CIRCLE'])

# AMT_REQ_CREDIT_BUREAU_HOUR => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_HOUR'] = round(df_result['AMT_REQ_CREDIT_BUREAU_HOUR'])

# AMT_REQ_CREDIT_BUREAU_DAY => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_DAY'] = round(df_result['AMT_REQ_CREDIT_BUREAU_DAY'])

# AMT_REQ_CREDIT_BUREAU_WEEK => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_WEEK'] = round(df_result['AMT_REQ_CREDIT_BUREAU_WEEK'])

# AMT_REQ_CREDIT_BUREAU_MON => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_MON'] = round(df_result['AMT_REQ_CREDIT_BUREAU_MON'])

# AMT_REQ_CREDIT_BUREAU_QRT => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_QRT'] = round(df_result['AMT_REQ_CREDIT_BUREAU_QRT'])

# AMT_REQ_CREDIT_BUREAU_YEAR => arrondir à l'entier le plus proche
df_result['AMT_REQ_CREDIT_BUREAU_YEAR'] = round(df_result['AMT_REQ_CREDIT_BUREAU_YEAR'])

# CAR_AGE_RANGE => arrondir à l'entier le plus proche
df_result['CAR_AGE_RANGE'] = round(df_result['CAR_AGE_RANGE'])

### Unification de la donnée Finale

#### On supprimes les anciennes colonnes avec les NaN

In [26]:
columnt_to_delet = df_result.columns.values.tolist()
dt_train = dt_train.drop(columnt_to_delet, axis = 1)

#### Ajout d'une colonnes de jointure à la table contenant la donnée sans NaN

In [27]:
index_id = []
for i in dt_train['SK_ID_CURR']:
    index_id.append(i)
    
df_result['SK_ID_CURR'] = index_id

#### Ajout des nouvelles colonnes sans NaN

In [28]:
dt_train = pd.merge(dt_train, df_result, on = 'SK_ID_CURR')

#### Traduction des valeur anciennement traduit en entiers

In [29]:
for i in range (len(values_save)) :
    for j in range (len(values_save[i])) :
        dt_train[values_save[i][0]] = dt_train[values_save[i][0]].replace(j,values_save[i][j])

#### Création des colonnes dépendant d'anciennes colonnes contenant des NaN 

In [30]:
# Réctification incohérence entre CAR_AGE_RANGE & FLAG_OWN_CAR
for i in range(len(dt_train['SK_ID_CURR'])) :
    if dt_train['FLAG_OWN_CAR'][i] == 'N' :
        dt_train['CAR_AGE_RANGE'][i] = 0

# CREDIT_PER_ON_GOODPRICE : somme du bien souhaité crédité
dt_train['CREDIT_PER_ON_GOODPRICE'] = round(((dt_train['AMT_CREDIT'] / dt_train['AMT_GOODS_PRICE']) * 100),2)

# TOTAL_AMT_CREDIT : somme totale à rembourser
dt_train['TOTAL_AMT_CREDIT'] = dt_train['AMT_CREDIT'] + dt_train['AMT_ANNUITY']

# TOTAL_AMT_REQ_CREDIT_BUREAU : nombre total de demandes à propos du crédit au bureau, avant son application
dt_train['TOTAL_AMT_REQ_CREDIT_BUREAU'] = dt_train['AMT_REQ_CREDIT_BUREAU_HOUR'] + dt_train['AMT_REQ_CREDIT_BUREAU_DAY'] + dt_train['AMT_REQ_CREDIT_BUREAU_WEEK'] + dt_train['AMT_REQ_CREDIT_BUREAU_MON'] + dt_train['AMT_REQ_CREDIT_BUREAU_QRT'] + dt_train['AMT_REQ_CREDIT_BUREAU_YEAR']

In [31]:
# CREDIT_PER_ON_GOODPRICE : somme du bien souhaité crédité
dt_train['PER_ANNUITY'] = round(((dt_train['AMT_ANNUITY'] / dt_train['TOTAL_AMT_CREDIT']) * 100),2)

In [32]:
dt_train = dt_train.drop(['AMT_CREDIT','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT' ,'AMT_REQ_CREDIT_BUREAU_YEAR'], axis = 1)

### Analyse des données après nettoyage

In [33]:
incomplet_data = 0
for i in dt_train.columns.values.tolist() :      
    incomplet_data += dt_train[i].isna().sum()


total_info = len(dt_train['SK_ID_CURR']) * len(dt_columns)

print(f"Total Data = {total_info}")
print(f"Missing Data = {incomplet_data}")
print(f"Missing Data Rate = {round( (incomplet_data / total_info),2) * 100} %")

Total Data = 15990260
Missing Data = 0
Missing Data Rate = 0.0 %


In [34]:
dt_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,...,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,CAR_AGE_RANGE,CREDIT_PER_ON_GOODPRICE,TOTAL_AMT_CREDIT,TOTAL_AMT_REQ_CREDIT_BUREAU,PER_ANNUITY
0,100002,1,Cash loans,M,0,1,0,202500.0,Working,Secondary,...,0.139376,2.0,2.0,2.0,2.0,2.0,115.84,431298.0,1.0,5.73
1,100003,0,Cash loans,F,0,0,0,270000.0,State servant,Higher education,...,0.548131,1.0,0.0,1.0,0.0,2.0,114.52,1329201.0,0.0,2.69
2,100004,0,Revolving loans,M,1,1,0,67500.0,Working,Secondary,...,0.729567,0.0,0.0,0.0,0.0,3.0,100.00,141750.0,0.0,4.76
3,100006,0,Cash loans,F,0,1,0,135000.0,Working,Secondary,...,0.506965,2.0,0.0,2.0,0.0,2.0,105.28,342369.0,2.0,8.67
4,100007,0,Cash loans,M,0,1,0,121500.0,Working,Secondary,...,0.452414,0.0,0.0,0.0,0.0,2.0,100.00,534865.5,0.0,4.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307500,456251,0,Cash loans,M,0,0,0,157500.0,Working,Secondary,...,0.540520,0.0,0.0,0.0,0.0,1.0,113.20,282258.0,1.0,9.76
307501,456252,0,Cash loans,F,0,1,0,72000.0,Pensioner,Secondary,...,0.448786,0.0,0.0,0.0,0.0,1.0,119.80,281551.5,2.0,4.26
307502,456253,0,Cash loans,F,0,1,0,153000.0,Working,Higher education,...,0.218859,6.0,0.0,6.0,0.0,2.0,115.84,707643.0,3.0,4.24
307503,456254,1,Cash loans,F,0,1,0,171000.0,Commercial associate,Secondary,...,0.661024,0.0,0.0,0.0,0.0,2.0,115.84,390312.0,0.0,5.18


# Export du résultat final

In [48]:
dt_train.to_csv('Data_Base_Enhanced/train_enhanced.csv', index = False)